### Student Information
Name: 許詠晴

Student ID: 111062575

GitHub ID: Hsu0623

Kaggle name: Hsu1004

Kaggle private scoreboard snapshot:0.40672

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

In [2]:
### Begin Assignment Here

# First: Do the take home exercises in the DM2022-Lab2-master Repo
#### address of DM2022-Lab2-master:  https://github.com/Hsu0623/DM2022-Lab2-Master/blob/main/DM2022-Lab2-Master.ipynb 

# Second: 

Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the img folder of this repository and rerun the cell Student Information.

OK!

# Third:

Below is my code and comment.

This code runs in Google colab (https://colab.research.google.com/drive/1vSvG0GHHc5UI68YyXMggwcnU4BVHwPjL#scrollTo=74UHu7rBXCwy), because my PC is poor.

However, I'm a free number on Google Colab and am plagued by RAM limitations.

So, I had to run some cells, save the current state (save data and output as .json, .pickle, etc.), restart 
the kernel, read the saved file, and rerun the cells, repeatly and repeatly again.

### Code start

In [ ]:
# import needed module firstly
import json
import numpy as np
from glob import glob
import tensorflow as tf
import pandas as pd
import csv

### read json file
* read tweets_DM.json
  * only read tweet_id and text.

In [ ]:
file_path = "/content/drive/MyDrive/data_mining/lab2/tweets_DM.json"

ids = []
texts = []

with open(file_path, 'r') as fp:
  for line in fp:
    data = json.loads(line)
    ids.append(data["_source"]["tweet"]["tweet_id"])
    texts.append(data["_source"]["tweet"]["text"])

df = pd.DataFrame(list(zip(ids,texts)), columns=["tweet_id", "text"])

* read data_identification.csv in order to seperate data into train and test

In [ ]:
file_path = "/content/drive/MyDrive/data_mining/lab2/data_identification.csv"

with open(file_path, 'r') as fp:
  df_id = pd.read_csv(fp)

* read emotion.csv as train label

In [ ]:
file_path = "/content/drive/MyDrive/data_mining/lab2/emotion.csv"

with open(file_path, 'r') as fp:
  df_emotion = pd.read_csv(fp)

In [ ]:
df_emotion["emotion"].value_counts()

* merge data and identification by tweet_id

In [ ]:
df = pd.merge(df, df_id, on="tweet_id")

* seperate data into train and test

In [ ]:
grouped = df.groupby(df.identification)
df_train = grouped.get_group("train")
df_test = grouped.get_group("test")

In [ ]:
print(len(df_train))
print(len(df_emotion))

* finally, merge emotion label and train_data

In [ ]:
df_train = pd.merge(df_train, df_emotion, on="tweet_id")

* eliminate identification because it is unuseful

In [ ]:
df_train.pop("identification")
df_test.pop("identification")

* show train data

In [ ]:
df_train

### data preprocessing
* using nltk.tokenize to tokenize setence(data[text])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

df_train["tokens"] = df_train["text"].apply(lambda x: word_tokenize(x))
df_test["tokens"] = df_test["text"].apply(lambda x: word_tokenize(x))

In [ ]:
print(len(df_train))
print(len(df_test))

* save data 

In [ ]:
df_train.to_pickle("/content/drive/MyDrive/data_mining/lab2/df_train.pkl")
df_test.to_pickle("/content/drive/MyDrive/data_mining/lab2/df_test.pkl")

#### restart kernel and load data
* because executing above codes spend lost of RAM and time

In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/data_mining/lab2/df_train.pkl")

test_df = pd.read_pickle("/content/drive/MyDrive/data_mining/lab2/df_test.pkl")

In [ ]:
print(len(df_train))
print(len(train_df))
print(len(df_test))
print(len(test_df))

#### continue to data preprocess

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### using embedding to change the type of X_train/X_test(string) to number.
* because the type of the input of deep leraning model needs number 

* firstly, Onehot encoding emotion(Y_train) 

In [ ]:
Y_train = train_df["emotion"]
print("Before encoding label:")
print(Y_train[:5])

print("After encoding:")
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y_train)
print(integer_encoded[:5])
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y_train_onehot = onehot_encoder.fit_transform(integer_encoded)
print(Y_train_onehot[:5])


* save Y_train_onehot endcoding file 

In [ ]:
np.save("/content/drive/MyDrive/data_mining/lab2/Y_train_onehot.npy", Y_train_onehot)

In [ ]:
Y_train_onehot.shape

#### training w2v model with X_train data only

In [ ]:
X_train = train_df["tokens"]

In [ ]:
from gensim.models import Word2Vec

## setting
vector_dim = 150
window_size = 3
min_count = 1
training_epochs = 10

## model
w2v_model = Word2Vec(sentences=X_train, size=vector_dim, window=window_size, min_count=min_count, iter=training_epochs)
print("Word2Vec trains OK")

* save w2v weights and result

In [ ]:
w2v_model.save("/content/drive/MyDrive/data_mining/lab2/word2vec.model")

* load if w2v_model is pretrained

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec.load("/content/drive/MyDrive/data_mining/lab2/word2vec.model")

In [ ]:
print(train_df.shape)

* convert all string data into word vector and save them

In [ ]:
import pickle

#### turn each word into vector and save
* Due to the limitation of RAM, I have to split 1.4 million training data into 15 pieces (100,000 pieces each)

In [ ]:
# X_w2v_train = np.array([np.mean(w2v_model.wv[string],axis=0) for string in X_train[1200000:1300000]])
# X_w2v_train = np.array([np.mean(w2v_model.wv[string],axis=0) for string in X_train[1300000:1400000]])
X_w2v_train = np.array([np.mean(w2v_model.wv[string],axis=0) for string in X_train[1400000:]])

In [ ]:
with open("/content/drive/MyDrive/data_mining/lab2/X_w2v_train_1455563.pickle", "wb") as fp:   #Pickling
  pickle.dump(X_w2v_train, fp)

#### turn each testing word into vectors and save

Althouh I have all word vectors of data, the memory size usage is tooooooo big

Thus, averaging each word vectors of a setence as the representation of the setence

And save the average files into a file

In [ ]:
X_test = test_df["tokens"]

* When a test word is encountered that does not appear in the training words, it is replaced with a zero vector.

In [ ]:
X_w2v_test = []

for string in X_test[400000:]:
  tmp = np.array([])
  for word in string:
    try:
      vec = w2v_model.wv[word]
    except:
      vec = np.zeros((1,150))
    if tmp.shape==(0,):
      tmp = vec
    else:
      tmp=np.vstack((tmp,vec))
  X_w2v_test.append(np.mean(tmp,axis=0))


X_w2v_test = np.array(X_w2v_test)

In [ ]:
with open("/content/drive/MyDrive/data_mining/lab2/X_w2v_test_411972.pickle", "wb") as fp:   #Pickling
  pickle.dump(X_w2v_test, fp)

### finally, the data preprocessing is finished.
the shape of X_train is (1455563, 150)

the shape of X_test is (411972, 150)

## start to classify which emotion the setence is

* restrat kernel because the limitation of RAM
#### load all required data 

In [ ]:
import json
import numpy as np
from glob import glob
import tensorflow as tf
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle

* load X_train, Y_train, and X_test

In [ ]:
with open("/content/drive/MyDrive/data_mining/lab2/X_avg_train.pickle","rb") as fp:   #Pickling
  X_train = pickle.load(fp)

Y_train_onehot = np.load("/content/drive/MyDrive/data_mining/lab2/Y_train_onehot.npy")

print(X_train.shape)
print(Y_train_onehot.shape)

In [ ]:
with open("/content/drive/MyDrive/data_mining/lab2/X_avg_test.pickle","rb") as fp:   #Pickling
  X_test = pickle.load(fp)


print(X_test.shape)


## deep learning Dense -  build classifier model

In [ ]:
input = tf.keras.layers.Input(shape=(150))
x = tf.keras.layers.Dense(128, activation='relu')(input)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(8, activation='softmax')(x)

cls_model = tf.keras.models.Model(inputs=input,outputs=output)

cls_model.compile(optimizer='adam',
         loss='categorical_crossentropy',
         metrics=['accuracy'])

cls_model.summary()

#### training model

In [ ]:
# training setting
epochs = 25
batch_size = 64

# training!
history = cls_model.fit(X_train, Y_train_onehot, epochs=epochs, batch_size=batch_size)
print('training finish')

### predict test data

In [ ]:
pred_result = cls_model.predict(X_test)

### save the predict result into csv

* turn oneHot encoding into emtion(string)

In [ ]:
map = ["anger","anticipation","digust","fear","joy","sadness","surprise","trust"]
pred_classes = [map[np.argmax(pred, axis=None, out=None)] for pred in pred_result]

* getting tweet_ids and combine it with emtions

In [ ]:
test_id = pd.read_pickle("/content/drive/MyDrive/data_mining/lab2/df_test.pkl")
test_id = test_id["tweet_id"].tolist()

* write predict result into csv

In [ ]:
import csv
with open("/content/drive/MyDrive/data_mining/lab2/pre_test0.csv", "w") as fp:
  write = csv.writer(fp)
  write.writerows([("id","emotion")])
  write.writerows(zip(test_id,pred_classes))